Per il preprocessing serve fare una pipeline in cui:
- vengono gestiti i valori Nan
- viene fatto encoding o standardizzazione

Pipeline diverse per tipi di dato diversi

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pandas.api.types import CategoricalDtype
from scipy.stats import chi2_contingency
import seaborn as sns


df = pd.read_csv('data.csv')


In [2]:


binary_cols = ['HLTHPLN1','MEDCOST','TOLDHI2','CVDINFR4',
            'CVDCRHD4','CVDSTRK3','ASTHMA3','CHCSCNCR','CHCOCNCR',
            'CHCCOPD1','HAVARTH3','ADDEPEV2','CHCKIDNY','VETERAN3',
            'INTERNET','QLACTLM2','USEEQUIP','BLIND','DECIDE',
            'DIFFWALK','DIFFDRES','DIFFALON','SMOKE100','EXERANY2','FLUSHOT6',
            'PNEUVAC3', 'HIVTST6', 'PERSDOC2', 'BPHIGH4'] 

nominal_cols = [
    'MARITAL','EMPLOY1','_RACE', '_BMI5CAT'
]

numeric_cols = [
    'PHYSHLTH','MENTHLTH','CHILDREN','NUMADULT_2',
    'ALCDAY5', 'FRUITJU1','NutritionScore',  'STRENGTH'
]

ordinal = {
    'GENHLTH':       [1,2,3,4,5],       # 1=Excellent … 5=Poor
    'CHECKUP1':      [1,2,3,4,5,6,7,8], # 
    'CHOLCHK':       [1,2,3,4],         
    '_AGE_G':        [1,2,3,4,5,6],     # 1=18–24 … 6=65+
    '_PACAT1':       [1,2,3,4],         # 1=High active … 4=Inactive     
    'EDUCA':         [1,2,3,4,5,6],     # 1=Less than high school … 6=Post-grad
    'INCOME2':       [1,2,3,4,5,6,7,8], # 1=Less than $10K … 8=$75K or more 
    '_SMOKER3':      [1,2,3,4],         # 1=Current every day … 4=Never
}

"""
# 2) Cast nominali a 'category' (manteniamo i codici numerici)
for col in nominal_cols:
    df[col] = df[col].astype('category')

# 3) Cast ordinali a 'category' con ordered=True
for col, cats in ordinal_asc.items():
    cat_type = CategoricalDtype(categories=cats, ordered=True)
    df[col] = df[col].astype(cat_type)

for col, cats in ordinal_desc.items():
    cat_type = CategoricalDtype(categories=cats, ordered=True)
    df[col] = df[col].astype(cat_type)
"""
# Nominali: stringa coerente
for col in nominal_cols:
    df[col] = df[col].astype(str)
    df[col] = df[col].replace(['nan', 'None', '-9223372036854775808'], np.nan)

# Ordinali: forzati a float con NaN se conversione fallisce
for col in list(ordinal.keys()):
    df[col] = pd.to_numeric(df[col], errors='coerce')


df[binary_cols] = df[binary_cols].astype('Int64')

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')


df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321457 entries, 0 to 321456
Data columns (total 50 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   GENHLTH         320707 non-null  float64
 1   PHYSHLTH        315181 non-null  float64
 2   MENTHLTH        316891 non-null  float64
 3   HLTHPLN1        320798 non-null  Int64  
 4   PERSDOC2        320512 non-null  Int64  
 5   MEDCOST         320812 non-null  Int64  
 6   CHECKUP1        318370 non-null  float64
 7   BPHIGH4         320806 non-null  Int64  
 8   CHOLCHK         317268 non-null  float64
 9   TOLDHI2         318789 non-null  Int64  
 10  CVDINFR4        320086 non-null  Int64  
 11  CVDCRHD4        318652 non-null  Int64  
 12  CVDSTRK3        320613 non-null  Int64  
 13  ASTHMA3         320590 non-null  Int64  
 14  CHCSCNCR        320631 non-null  Int64  
 15  CHCOCNCR        320822 non-null  Int64  
 16  CHCCOPD1        319944 non-null  Int64  
 17  HAVARTH3  

In [3]:

from sklearn.model_selection import train_test_split
# Rimuove ogni tipo categorico residuo
for col in df.columns:
    if pd.api.types.is_categorical_dtype(df[col]):
        df[col] = df[col].astype(str)  # resetta tipo
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except:
            pass

# Definizione feature e target
X = df.drop(columns=['DIABETE3'])
y = df['DIABETE3']

# Split stratificato
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)


/var/folders/6v/1zb17jyj7kb910z5z0pnvx5c0000gn/T/ipykernel_63019/1465117484.py:4: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):


In [4]:
print(df.dtypes[df.dtypes == 'category'])         # devono essere 0
print(X.dtypes[X.dtypes == 'category'])           # devono essere 0


Series([], dtype: object)
Series([], dtype: object)


In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, confusion_matrix, roc_auc_score, roc_curve, auc)
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm  # ✅ Aggiunto tqdm per barre di avanzamento

import Preprocessor  # assicurati che questo sia definito correttamente
from Preprocessor import preprocessor


# === MODELLI ===
models = {
    'Random Forest': RandomForestClassifier(random_state=42)
    # Puoi scommentare altri modelli qui sotto se vuoi usarli:
    # 'Logistic Regression': LogisticRegression(max_iter=500),
    # 'AdaBoost': AdaBoostClassifier(random_state=42),
    # 'Decision Tree': DecisionTreeClassifier(random_state=42),
    # 'KNN': KNeighborsClassifier(),
    # 'Bayesian': GaussianNB(),
    # 'XGBoost': XGBClassifier(random_state=42)
}

# === STRATIFIED CV ===
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
n_classes = 3  # 0=no diabete, 1=prediabete, 2=diabete

model_scores = {}
roc_curves = {}
conf_matrices = {}
all_results = []

# === LOOP PER MODELLO CON TQDM ===
for model_name, model in tqdm(models.items(), desc="Valutazione modelli"):

    pipeline = Pipeline(steps=[
        ('preprocessing', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('model', model)
    ])
    y_true = []
    y_pred = []
    y_prob_all = []

    # LOOP CON TQDM SUI FOLD
    for fold_idx, (train_idx, test_idx) in enumerate(tqdm(skf.split(X_train, y_train), total=5, desc=f"📂 CV {model_name}")):
        X_fold_train, y_fold_train = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_fold_test,  y_fold_test  = X_train.iloc[test_idx],  y_train.iloc[test_idx]

        pipeline.fit(X_fold_train, y_fold_train)

        y_pred_fold = pipeline.predict(X_fold_test)
        y_prob_fold = pipeline.predict_proba(X_fold_test)

        y_true.extend(y_fold_test)
        y_pred.extend(y_pred_fold)
        y_prob_all.extend(y_prob_fold)

    # === METRICHE ===
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob_all = np.array(y_prob_all)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    roc_auc = roc_auc_score(label_binarize(y_true, classes=[0,1,2]),
                            y_prob_all, average='macro', multi_class='ovr')

    all_results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC': roc_auc
    })

    conf_matrix = confusion_matrix(y_true, y_pred)
    conf_matrices[model_name] = conf_matrix

    fpr = dict()
    tpr = dict()
    roc_auc_class = dict()
    y_true_bin = label_binarize(y_true, classes=[0,1,2])

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_prob_all[:, i])
        roc_auc_class[i] = auc(fpr[i], tpr[i])

    roc_curves[model_name] = (fpr, tpr, roc_auc_class)

# === RISULTATI ===
results_df = pd.DataFrame(all_results)
print(results_df)

# === ROC CURVES MULTICLASS ===
for model_name, (fpr, tpr, roc_auc_class) in roc_curves.items():
    plt.figure(figsize=(8, 6))
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc_class[i]:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f'ROC Curve - {model_name}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

# === CONFUSION MATRICES ===
num_models = len(models)
cols = 3
rows = num_models // cols + int(num_models % cols > 0)

fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
axes = axes.flatten()

for idx, (model_name, conf_matrix) in enumerate(conf_matrices.items()):
    ax = axes[idx]
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax)
    ax.set_title(f'{model_name}')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')

for idx in range(len(conf_matrices), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()


🔍 Valutazione modelli:   0%|          | 0/1 [00:52<?, ?it/s]


KeyboardInterrupt: 